Code to test ensemble mean

In [1]:
import numpy as np
import xarray as xr
import pandas as pd

# create coords
lon_vec = np.arange(-179.5, 180, 1)
lat_vec = np.arange(-89.5, 90, 1)
tim_vec = pd.date_range(start='2020-01-01',end='2020-12-31',freq='MS')

# create data
model1 = np.random.randn(len(tim_vec), len(lon_vec), len(lat_vec))
model2 = np.random.randn(len(tim_vec), len(lon_vec), len(lat_vec))
model3 = np.random.randn(len(tim_vec), len(lon_vec), len(lat_vec))

# put data into a dataset
ds = xr.Dataset(
    data_vars=dict(
        ens1=(["time", "lon", "lat"], model1),
        ens2=(["time","lon", "lat"], model2),
        ens3=(["time", "lon", "lat"], model3),
    ),
    coords=dict(
        lon=(["lon"], lon_vec),
        lat=(["lat"], lat_vec),
        time=(["time"], tim_vec),
    ),
    attrs=dict(description="Monthly temperature output from three different models"),
)

In [3]:
ds

<xarray.Dataset>
Dimensions:  (time: 12, lon: 360, lat: 180)
Coordinates:
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * time     (time) datetime64[ns] 2020-01-01 2020-02-01 ... 2020-12-01
Data variables:
    ens1     (time, lon, lat) float64 0.2234 0.2951 0.2024 ... -1.166 0.4533
    ens2     (time, lon, lat) float64 0.7229 -0.8233 -0.2411 ... -0.2781 0.5458
    ens3     (time, lon, lat) float64 0.3771 -0.2017 1.689 ... -0.4501 1.908
Attributes:
    description:  Monthly temperature output from three different models

In [5]:
variables = list(ds)

In [25]:
da_list = [ds[var].expand_dims('ensemble', axis=0).assign_coords(ensemble=[var]).rename('temperature') for var in variables]

In [34]:
ds_merged = xr.merge(da_list)
ds_merged

<xarray.Dataset>
Dimensions:      (lon: 360, lat: 180, time: 12, ensemble: 3)
Coordinates:
  * lon          (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat          (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time         (time) datetime64[ns] 2020-01-01 2020-02-01 ... 2020-12-01
  * ensemble     (ensemble) <U4 'ens1' 'ens2' 'ens3'
Data variables:
    temperature  (ensemble, time, lon, lat) float64 0.2234 0.2951 ... 1.908

In [35]:
global_attrs = ds.attrs
ds_merged.attrs = global_attrs

In [36]:
import clouddrift

In [41]:
import os
import inspect

In [42]:
print(os.path.abspath(inspect.getfile(clouddrift.RaggedArray)))

/Users/selipot/opt/miniconda3/envs/research/lib/python3.9/site-packages/clouddrift/dataformat.py
